In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import cv2

In [2]:
#VGG16
def VGG_lite(input_shape, num_classes):
    model = Sequential()

    # Block 1
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Classifier
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    return model


# Example usage:
input_shape = (224, 224, 3)  # Input shape of images
num_classes = 2  # Number of classes (binary classification)

model = VGG_lite(input_shape, num_classes)
model.summary()

C:\Users\Olexandr Perehinchuk\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 14, 14, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,415,170 (54.99 MB)

 Trainable params: 14,415,170 (54.99 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
import tensorflow as tf

# Allow gpu usage
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
try:
    tf.config.experimental.set_memory_growth = True
except Exception as ex:
    print(ex)

[]


In [4]:
def load_images_from_folder(folder_path, img_size=(224, 224)):
    images = []
    labels = []

    for filename in os.listdir(folder_path):
        label = filename  # Assuming file name format is label.image_number.jpg
        if label == "Cancer":
            label1 = 1 # Convert label to integer if necessary
        else:
            label1 = 0
        
        for photoname in os.listdir(folder_path + "/" + label):
            img = cv2.imread(os.path.join(folder_path, filename, photoname))
            img = cv2.resize(img, img_size)  # Resize image to desired dimensions
            images.append(img)
            labels.append(label1)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

In [5]:
imagesTrain, labelsTrain = load_images_from_folder("Train1")


In [6]:
imagesTest, labelsTest = load_images_from_folder("Test1")


In [7]:
imagesTrain, labelsTrain = load_images_from_folder("Train1")
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(imagesTrain, labelsTrain, epochs=10, validation_split=0.2)

Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.5336 - loss: 40.3901 - val_accuracy: 0.0000e+00 - val_loss: 1.0519
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.5897 - loss: 0.6725 - val_accuracy: 0.9888 - val_loss: 0.4453
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.5915 - loss: 0.6461 - val_accuracy: 0.2098 - val_loss: 0.8300
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.6525 - loss: 0.6275 - val_accuracy: 0.3147 - val_loss: 0.8316
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 88s 2s/step - accuracy: 0.6933 - loss: 0.6051 - val_accuracy: 0.2612 - val_loss: 1.2560
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 88s 2s/step - accuracy: 0.6525 - loss: 0.6359 - val_accuracy: 0.1607 - val_loss: 0.9156
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.6348 - loss: 0.6281 - val_accuracy: 0.1027 - val_loss: 0.9966
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.6182 - loss: 0.6489 - val_accuracy: 0.0759 - val_

In [8]:
test_loss, test_accuracy = model.evaluate(imagesTest, labelsTest)
print(f"Test Accuracy: {test_accuracy}")
test_loss1, test_accuracy1 = model.evaluate(imagesTrain, labelsTrain)
print(test_accuracy1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 366ms/step - accuracy: 0.4286 - loss: 1.4079
Test Accuracy: 0.347457617521286
70/70 ━━━━━━━━━━━━━━━━━━━━ 24s 343ms/step - accuracy: 0.7086 - loss: 0.6630
0.5082626342773438
